# Covid19 Data Processing
1. Data source is [John Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)
1. Convert wide format to long format and sanitize the data
1. Inputs are three files: `confimred`, `deaths`, and `recovered`
1. Ouput four files:
    1. `covid19-confirmed-<date>.csv`
    1. `covid19-deaths-<date>.csv`
    1. `covid19-recovered-<date>.csv`
    1. `covid19-<date>.csv` - contains confirmed, deaths and recovered
    1. `covid19.c6p` - geo-coordinates, i.e State to latlong

In [132]:
import pandas as pd

In [133]:
confirmed = pd.read_csv('../data-jhu/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths = pd.read_csv('../data-jhu/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
recovered = pd.read_csv('../data-jhu/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

# Convert wide format to long format

In [134]:
def wide_to_long(df, col):
    return pd.melt(df, id_vars=df.columns[:4], value_vars=df.columns[4:], var_name='Date', value_name=col)

# Sanitize data

In [135]:
def sanitize(df, col):
    df = wide_to_long(df, col=col)
    
    # rename
    df.rename(columns={'Province/State': 'Province_State', 'Country/Region':'Country_Region'}, inplace=True)
    
    # remove comma
    df['Country_Region']=df['Country_Region'].str.replace(',', '')
    df['Province_State']=df['Province_State'].str.replace(',', '')
    
    # change date format
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')
    
    return df;

In [136]:
confirmed = sanitize(confirmed, 'ConfirmedCases')
deaths = sanitize(deaths, 'Fatalities')
recovered = sanitize(recovered, 'RecoveredCases')

In [137]:
def fill_state(df, src='Country_Region', tgt='Province_State'):
    """
    Use the value in `src` column to fill the `tgt` column if `tgt` is NaN
    """
    idx = df[tgt].isnull()
    df.loc[idx, tgt] = df[src].loc[idx]

In [138]:
fill_state(confirmed)
fill_state(deaths)
fill_state(recovered)

In [139]:
confirmed.shape

(28920, 6)

In [140]:
confirmed.head()

,Province_State,Country_Region,Lat,Long,Date,ConfirmedCases
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,2
1,Japan,Japan,36.0000,138.0000,01/22/2020,2
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0


In [141]:
deaths.head()

,Province_State,Country_Region,Lat,Long,Date,Fatalities
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,0
1,Japan,Japan,36.0000,138.0000,01/22/2020,0
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0


In [142]:
recovered.head()

,Province_State,Country_Region,Lat,Long,Date,RecoveredCases
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,0
1,Japan,Japan,36.0000,138.0000,01/22/2020,0
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0


In [166]:
# merge confirmed, deaths and recovered
merged = pd.DataFrame()
for df, col in zip([confirmed, deaths, recovered], ['ConfirmedCases', 'Fatalities', 'RecoveredCases']):
    merged = df if len(merged)==0 else pd.concat([merged, df[col]], axis=1)

In [165]:
merged.head()

,Province_State,Country_Region,Lat,Long,Date,ConfirmedCases,Fatalities,RecoveredCases
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,2,0,0
1,Japan,Japan,36.0000,138.0000,01/22/2020,2,0,0
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0,0,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0,0,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0,0,0


In [158]:
def write_data(df, filename='covid19.csv', exclude_cols=['Lat', 'Long']):
    df.drop(exclude_cols, axis=1).to_csv(filename, index=False)

In [168]:
date=pd.to_datetime(confirmed['Date'].max()).strftime('%m.%d.%y')
write_data(confirmed, 'covid19-confirmed-{}.csv'.format(date))
write_data(deaths, 'covid19-deaths-{}.csv'.format(date))
write_data(recovered, 'covid19-recovered-{}.csv'.format(date))
write_data(merged, 'covid19-{}.csv'.format(date))

In [145]:
def write_geo(data, filename='covid19.c6p', cols=['Province_State', 'Lat', 'Long']):
    data[cols].to_csv(filename, index=False)

In [146]:
write_geo(confirmed, 'covid19.c6p')